In [5]:
import pandas as pd
import numpy as np
from pathlib import Path
import statsmodels.api as sm
import warnings

warnings.filterwarnings('ignore')

# 1. 参数设置与数据加载

In [6]:
root_path = Path('../')
data_path = root_path / 'data' / 'final_data_panel.pkl'

proxys = ['beta_MKT', 'retvol', 'ivol', 'CFVOL', 'age', 'disp', 'cgo']
portfolio_returns = {}

data = pd.read_pickle(data_path)
data

,date,asset,beta_MKT,ivol,retvol,age,disp,CFVOL,LOGBM,LOGME,mom_minus1_0,mom_minus12_minus1,mom_minus36_minus12,mom_minus11_minus2,cgo,turnover,future_return,score
0,2014-01,000006,0.714892,-0.385213,-0.253707,-0.698990,NaN,-0.195242,0.852785,-0.290561,-1.010538,-0.434441,1.405009,0.228706,-0.437893,-0.381935,-0.016018,-1.735858
1,2014-01,000021,0.902618,-0.210387,0.403084,-0.672945,NaN,0.330226,0.302500,0.840230,2.201666,0.015993,-1.109247,0.205026,0.858371,-0.227697,-0.077961,-0.956005
2,2014-01,000028,-0.789488,0.177394,0.003037,-0.682054,NaN,-0.804418,-1.637611,1.817995,1.169076,0.485133,1.219088,0.203879,2.514612,-0.564752,-0.078858,1.029076
3,2014-01,000030,-0.926812,3.139430,3.924975,-0.680567,NaN,0.162950,0.281369,0.279086,-0.336719,-1.182369,0.487879,-1.298161,-0.311534,-0.033620,0.255537,1.454450
4,2014-01,000031,0.503439,-0.779173,-0.700685,-0.679068,NaN,-0.857062,0.960635,-0.096190,-0.409915,-0.677725,-0.352625,-0.694764,-0.952216,-0.911117,-0.090141,-0.388839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57398,2023-12,688772,NaN,NaN,1.188984,2.479040,1.459955,-0.483142,-0.825818,0.268728,1.004084,0.500160,NaN,-0.035467,1.150507,0.641422,-0.390277,0.647214
57399,2023-12,688778,NaN,NaN,0.525038,2.227011,0.561135,0.007475,0.054129,-0.952300,0.288576,-0.910079,NaN,-0.952374,-0.640407,-0.292542,-0.240666,0.714458
57400,2023-12,688779,NaN,NaN,-0.225561,2.227011,1.209956,0.853260,0.108926,-1.478866,0.033653,-1.592488,NaN,-1.541132,-2.424496,0.662592,-0.230137,0.961017
57401,2023-12,688819,-0.840286,NaN,0.298146,1.578404,0.809786,0.458108,0.156839,0.563051,-0.531444,-0.587884,NaN,-0.364451,-0.494163,0.508415,-0.116046,0.198923


# 2. 辅助函数定义
# - `one_month`：对单月数据按代理变量五分位分组，返回各组未来收益及多空组合（Q5-Q1）。
# - `calculate_nw_t`：计算单序列的 Newey-West t 值。
# - `mean_and_nw_t`：汇总平均收益与 t 值。

In [7]:
def one_month(group: pd.DataFrame, sort_col: str) -> pd.Series:
    """返回某个月按 sort_col 分为五组后的等权收益（含 Q5-Q1）"""
    if group.shape[0] < 5:
        return pd.Series(dtype=float)
    g = group.copy()
    g['proxy_rank'] = pd.qcut(g[sort_col], 5, labels=False) + 1
    ret = g.groupby('proxy_rank')['future_return'].mean()
    ret.index = [f'Q{i}' for i in ret.index]
    ret['LS'] = ret['Q5'] - ret['Q1']
    return ret

def calculate_nw_t(series: pd.Series, lags: int = 6) -> float:
    """
    对单个序列计算 Newey-West T 统计量。
    原理：对常数项进行 OLS 回归，并使用 HAC (Heteroskedasticity and Autocorrelation Consistent) 协方差矩阵。
    """
    # 去除空值
    y = series.dropna()

    # 如果数据太少，无法回归，返回 NaN
    if len(y) < lags + 2:
        return np.nan

    # 自变量 X 为常数 1 (截距项)
    X = np.ones((len(y), 1))

    try:
        # 建立 OLS 模型
        model = sm.OLS(y, X)

        # fit() 时指定 cov_type='HAC' 开启 Newey-West 调整
        # maxlags=6 是月度数据常用的滞后阶数
        results = model.fit(cov_type='HAC', cov_kwds={'maxlags': lags})

        # 返回截距项的 t 值
        return results.tvalues[0]
    except Exception as e:
        # 如果计算出错（例如数据全为0），返回 NaN
        return np.nan

def mean_and_nw_t(ret_df: pd.DataFrame) -> pd.DataFrame:
    """
    输入一个包含多列收益率的 DataFrame
    输出均值和 Newey-West T 统计量
    """
    # 1. 计算均值
    mean_ret = ret_df.mean()

    # 2. 计算 Newey-West T 值 (对每一列应用上面的 calculate_nw_t 函数)
    # 注意：这里调用了上面定义的函数
    nw_t_stat = ret_df.apply(lambda x: calculate_nw_t(x, lags=6))

    return pd.DataFrame({'mean': mean_ret, 't_stat_NW': nw_t_stat})

# 3. 单变量排序 & 组合收益计算

In [8]:
'''1. 开始构建投资组合收益率 (Portfolio Sorts)...'''

for proxy in proxys:
    # 检查列是否存在于 data 中
    if proxy not in data.columns:
        print(f"警告: {proxy} 不在数据列中，跳过。")
        continue

    cols = ['date', 'asset', proxy, 'future_return']
    # 提取子集并去空值
    tmp = data[cols].dropna().copy()

    # 按月分组计算收益
    # group_keys=False 保持索引整洁
    monthly = tmp.groupby('date', group_keys=False).apply(lambda g: one_month(g, proxy))

    # 存入字典
    portfolio_returns[proxy] = monthly
    print(f"   - {proxy} 处理完成")

'''2. 开始计算 Newey-West T 统计量...'''

stats = {}

for proxy, ret_df in portfolio_returns.items():
    if not ret_df.empty:
        stats[proxy] = mean_and_nw_t(ret_df)

print("\n3. 计算结果展示：\n")
print("="*50)

for proxy, res in stats.items():
    print(f'Proxy: {proxy}')
    print(res)
    print("-" * 50)

   - beta_MKT 处理完成
   - retvol 处理完成
   - ivol 处理完成
   - CFVOL 处理完成
   - age 处理完成
   - disp 处理完成
   - cgo 处理完成

3. 计算结果展示：

Proxy: beta_MKT
                   mean  t_stat_NW
future_return                     
Q1             0.007489   1.188044
Q2             0.009102   1.399805
Q3             0.007367   1.130828
Q4             0.006966   1.031137
Q5             0.004722   0.756894
LS            -0.002767  -1.054746
--------------------------------------------------
Proxy: retvol
                   mean  t_stat_NW
future_return                     
Q1             0.012572   2.125622
Q2             0.008387   1.260927
Q3             0.006828   1.046977
Q4             0.005250   0.770387
Q5             0.001754   0.257760
LS            -0.010818  -3.148816
--------------------------------------------------
Proxy: ivol
                   mean  t_stat_NW
future_return                     
Q1             0.013261   2.101198
Q2             0.008790   1.382284
Q3             0.008049   1.19421